In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import  preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold,StratifiedKFold
import matplotlib.pyplot as plt

In [3]:
#try xgboost
#fucntion from SRK
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, \
     seed_val=0, early_stop = 20,num_rounds=10000, eta = 0.1,\
     max_depth = 6,cv_dict = None,verbose_eval=True):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = eta
    param['max_depth'] = max_depth
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y,feature_names=feature_names)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y,feature_names=feature_names)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist,\
        early_stopping_rounds=early_stop,evals_result = cv_dict,verbose_eval = verbose_eval)
    else:
        xgtest = xgb.DMatrix(test_X,feature_names=feature_names)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

class CVstatistics(object):
    
    """
    self.result : the result dataframe storing the cv results
    self.endpoint : the first ending point for the validations
    self.turns: the turns for each validation
    
    validCurve : plot the validation curve,stop at the first endpoint
    """
    def __init__(self,result_dict,metric,k=5):
        self.metric = metric
        if type(result_dict) == pd.DataFrame:
            self.result = result_dict
        else:
            tempDict = {}
            for phase in ['train','test']:
                for turn in range(k):
                    tempDict[phase+str(turn)]=cv_result[turn][phase][metric]
                    self.result=pd.DataFrame(dict([ (key,pd.Series(v)) for key,v in tempDict.iteritems()]))    
        
        self.endpoint =len(self.result.filter(like = 'train').dropna())
        
        self.turns = self.result.filter(like = 'test').\
            apply(lambda x : ~np.isnan(x)).cumsum(axis=0).iloc[len(self.result)-1,:]

    def validCurve(self,start=0,stop_at_first = True):
        if stop_at_first:
            eout = self.result.filter(like = 'test').dropna().mean(axis=1)
            ein =  self.result.filter(like = 'train').dropna().mean(axis=1)
        else:
            eout = self.result.filter(like = 'test').mean(axis=1)
            ein =  self.result.filter(like = 'train').mean(axis=1)
        plt.plot(range(len(eout)), eout,
        range(len(ein)), ein)
        plt.xlabel("turn")
        plt.ylabel(self.metric)
        plt.title('Validation Curve')
        
        plt.show()
    
    def errorsAt(self,turn):
        eout = self.result.filter(like = 'test').loc[turn].mean()
        ein = self.result.filter(like = 'train').loc[turn].mean()
        return eout,ein
    

def showImportance(model,factor_name):
    factors = model.get_score(importance_type=factor_name)
    factor_list = []
    total = sum(factors.values())
    for key in factors:
        factors[key] = factors[key]*1.0/total
        factor_list.append((key,factors[key]))
    return sorted(factor_list,key=lambda x : x[1],reverse=True)
    


In [4]:
#from "this is a lit"s python version by rakhlin
def singleValueConvert(df1,df2,column,minimum_size=5):
    ps = df1[column].append(df2[column])
    grouped = ps.groupby(ps).size().to_frame().rename(columns={0: "size"})
    df1.loc[df1.join(grouped, on=column, how="left")["size"] <= minimum_size, column] = -1
    df2.loc[df2.join(grouped, on=column, how="left")["size"] <= minimum_size, column] = -1
    return df1, df2

In [24]:
def hcc_scoring(train_df,test_df,feature,labelValue,randomize=0.01,k=5,f=1,g=1,unrank_threshold =1,update_df =None):    
    #input is the train dataframe with its labels mapped to dummies
    #such as:
    tempTrain = train_df.join(pd.get_dummies(train_df[u'interest_level']).astype(int))
    
    new_feature = '_'.join(['hcc',feature,labelValue])
    
    #take the mean  for the feature on the given featureValue which is mapped to dummies
    prob = tempTrain[labelValue].mean()
    
    #take the mean and count for each feature value
    grouped = tempTrain.groupby(feature)[labelValue].agg({'count':'size','mean':'mean'})
    
    #perform the transform for lambda and the final score
    grouped["lambda"] = g / (g + np.exp((k - grouped["count"]) / f))
    grouped[new_feature] = grouped['lambda']*grouped['mean']+(1-grouped['lambda'])*prob
    
    #get the average score for the unrank features and reset them to this average
    unrankedMean = grouped.ix[grouped['count']<unrank_threshold,new_feature].mean()
    grouped.ix[grouped['count']<unrank_threshold,new_feature] = unrankedMean
    #grouped = grouped.reset_index()
    
    #adding to the test_df
    update_value  = test_df[[feature]].join(grouped,on = feature,how='left')[new_feature].fillna(prob)
    
    #adding some noise to the new 
    if randomize : update_value *= np.random.uniform(1 - randomize, 1 + randomize, len(test_df))
        
    #print 'New feature added:'+new_feature

    if update_df is None:
        update_df = test_df
    if new_feature not in update_df.columns: 
        update_df[new_feature] = np.nan
        
    update_df.update(update_value)


In [39]:
def performance_eval(train_df,test_df,feature,k,g=1,f=1,update_df =None,random = None):
    target_num_map = {'High':2, 'Medium':1, 'Low':0}
    temp=pd.concat([train_df[feature],pd.get_dummies(train_df.interest_level)], axis = 1)\
         .groupby(feature).mean()
     
    new_feature = feature+'_perf'
    
    temp.columns = ['tempHigh','tempLow', 'tempMed']
    
    temp['count'] = train_df.groupby(feature).count().iloc[:,1]
    temp["lambda"] = g / (g + np.exp((k - temp["count"] )/f))
    temp[feature+'_origin'] = temp['tempHigh']*4 + temp['tempMed']
    mean_values = temp.loc[:, feature+'_origin'].mean()
    
    temp[new_feature] = temp["lambda"]*temp[feature+'_origin']+(1-temp["lambda"])*mean_values    
    
    # Add uniform noise. Not mentioned in original paper.adding to each manager
    if random:
        temp[new_feature] *= np.random.uniform(1 - random, 1 + random, len(temp))     

    value = test_df[[feature]].join(temp, on=feature, how="left")[new_feature].fillna(mean_values)
    
    if update_df is None: update_df = test_df
    if new_feature not in update_df.columns: update_df[new_feature] = np.nan
    update_df.update(value)

In [7]:
#lodaing data
data_path = "../../kaggleData/2sigma/"
train_file = data_path + "train.json"
test_file = data_path + "test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)


(49352, 15)
(74659, 14)


In [8]:
#basic numerical features
features_to_use  = ["bathrooms", "bedrooms", "latitude", "longitude", "price"]

In [9]:
#some transfromed features
# count of photos #
train_df["num_photos"] = train_df["photos"].apply(len)

# count of "features" #
train_df["num_features"] = train_df["features"].apply(len)

# count of words present in description column #
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))

# convert the created column to datetime object so as to extract more features 
train_df["created"] = pd.to_datetime(train_df["created"])

# Let us extract some features like year, month, day, hour from date columns #
train_df["created_year"] = train_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
train_df["created_hour"] = train_df["created"].dt.hour

#some new numerical features related to the price
train_df["price_per_bath"] =  (train_df["price"]*1.0/train_df["bathrooms"]).replace(np.Inf,-1)
train_df["price_per_bed"] = (train_df["price"]*1.0/train_df["bedrooms"]).replace(np.Inf,-1)
train_df["bath_per_bed"] = (train_df["bathrooms"]*1.0/train_df["bedrooms"]).replace(np.Inf,-1)
train_df["price_per_room"] = (train_df["price"]*1.0/(train_df["bedrooms"]+train_df["bathrooms"])).replace(np.Inf,-1)

# adding all these new features to use list # "listing_id",
features_to_use.extend(["num_photos", "num_features", "num_description_words",\
                        "created_year","listing_id", "created_month", "created_day", "created_hour"])
#price new features
features_to_use.extend(["price_per_bed","bath_per_bed","price_per_room"])


In [10]:
train_df,test_df  = singleValueConvert(train_df,test_df,'manager_id',1)

In [41]:
features = list(features_to_use)
categorical = ["display_address", "street_address",'building_id','manager_id']
#categorical = ["display_address", "street_address",'building_id']
features.extend(categorical)
#the original version of using manager performance
features.append('manager_id_perf')
#the to be test version of using target prior
#features.append('hcc_manager_id_low')
#features.append('hcc_manager_id_medium')
#features.append('hcc_manager_id_high')

In [12]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_df),5,shuffle=True,random_state = 42)

In [42]:
#running and getting the cv from xgboost
cv_scores = []
cv_result = []

#K-FOLD already defined.If not ,use
#KF=KFold(len(train_X),5,shuffle=True,random_state = 42)
i=0
for dev_index, val_index in KF:
        result_dict = {}
                
        #split the orginal train set into dev_set and val_set
        dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
        
        """some preprocessing like feature constructed in cv manners"""
        skf=KFold(len(dev_set['interest_level']),5,shuffle=True,random_state = 42)
        #dev set adding manager skill
        for train,test in skf:
            performance_eval(dev_set.iloc[train,:],dev_set.iloc[test,:],feature='manager_id',k=5,g=10,
                           update_df = dev_set)
        performance_eval(dev_set,val_set,feature='manager_id',k=5,g=10)
        
        #or use the hcc encoding for target
        #hcc_scoring(train_df,test_df,feature,labelValue,randomize=0.01,k=5,f=1,g=1,unrank_threshold =5,update_df =None):    
        #for train,test in skf:
        #    hcc_scoring(dev_set.iloc[train,:],dev_set.iloc[test,:],feature='manager_id',labelValue='low',k=5,g=10,
        #                   update_df = dev_set)
        #    hcc_scoring(dev_set.iloc[train,:],dev_set.iloc[test,:],feature='manager_id',labelValue='high',k=5,g=10,
        #                   update_df = dev_set)
        #    hcc_scoring(dev_set.iloc[train,:],dev_set.iloc[test,:],feature='manager_id',labelValue='medium',k=5,g=10,
        #                   update_df = dev_set)
            
        #hcc_scoring(dev_set,val_set,feature='manager_id',labelValue='low',k=5,g=10)
        #hcc_scoring(dev_set,val_set,feature='manager_id',labelValue='high',k=5,g=10)
        #hcc_scoring(dev_set,val_set,feature='manager_id',labelValue='medium',k=5,g=10)

        for f in categorical:
            if dev_set[f].dtype=='object':
                #print(f)
                lbl = preprocessing.LabelEncoder()
                lbl.fit(list(dev_set[f])+list(val_set[f]))
                dev_set[f] = lbl.transform(list(dev_set[f].values))
                val_set[f] = lbl.transform(list(val_set[f].values))
        
        dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        
        """ 
         runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, \
         seed_val=0, early_stop = 20,num_rounds=10000, eta = 0.1,\
         max_depth = 6,cv_dict = None):
         """
        
        preds, model = runXGB(dev_X, dev_y, val_X, val_y,early_stop  = 20,\
                              feature_names = features,cv_dict = result_dict,verbose_eval=False)
        loss = log_loss(val_y, preds)
        cv_scores.append(loss)
        cv_result.append(result_dict)
        i+=1
        print 'loss for the turn '+str(i)+' is '+str(loss)

C:\Users\dell\Anaconda2\lib\site-packages\ipykernel\__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dell\Anaconda2\lib\site-packages\ipykernel\__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dell\Anaconda2\lib\site-packages\ipykernel\__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

loss for the turn 1 is 0.545069220844
loss for the turn 2 is 0.543712683933
loss for the turn 3 is 0.546600833117
loss for the turn 4 is 0.543309467742
loss for the turn 5 is 0.546859905985


In [37]:
#plot the validation curv
cvResult = CVstatistics(cv_result,'mlogloss')
cvResult.turns

test0    323
test1    261
test2    248
test3    249
test4    242
Name: 322, dtype: int32

In [43]:
#cvResult.validCurve(stop=False)
#some errors at certain turn to see the descending
cv_scores
np.mean(cv_scores)

0.54511042232399032

In [38]:
cvResult.errorsAt(200)

(0.5448403999999999, 0.4077706)

In [18]:
#show the importance of the features
showImportance(model,'gain')

[('manager_id_perf', 0.17154105330260128),
 ('price', 0.07508753888602825),
 ('bathrooms', 0.07241291362946572),
 ('price_per_bed', 0.06209520298109308),
 ('price_per_room', 0.06110715679412736),
 ('building_id', 0.056185268048130906),
 ('bedrooms', 0.049038003084418115),
 ('bath_per_bed', 0.04363033826639148),
 ('num_photos', 0.04341993987249756),
 ('longitude', 0.043082688310472836),
 ('num_features', 0.04062430799657446),
 ('latitude', 0.040552383501643356),
 ('created_hour', 0.0363196359557149),
 ('listing_id', 0.032045953842166144),
 ('street_address', 0.031899376268072994),
 ('num_description_words', 0.03162041016791853),
 ('display_address', 0.03149509532155309),
 ('manager_id', 0.029638093197271106),
 ('created_day', 0.024975231981100753),
 ('created_month', 0.023229408592758072)]